In [51]:
import requests
import pandas as pd
from datetime import datetime
import time

In [53]:
import requests

def fetch_event_and_markets(event_slug):
    """
    Fetches event data and associated market details from the Polymarket Gamma API.
    """

    base_url = "https://gamma-api.polymarket.com"

    # 1) Fetch event by slug
    event_url = f"{base_url}/events/slug/{event_slug}"
    event_resp = requests.get(event_url)

    if event_resp.status_code != 200:
        print(f"Failed to fetch event: {event_resp.status_code}")
        return

    event_data = event_resp.json()
    print("Event data:")
    print(event_data)

    # 2) Extract markets
    markets = event_data.get("markets", [])
    if not markets:
        print("No markets found for this event.")
        return

    print("\nMarkets found:", len(markets))
    
    # 3) For each market, fetch detailed market info
    for m in markets:
        market_slug = m.get("slug")
        if not market_slug:
            print("No slug for market, skipping.")
            continue

        market_url = f"{base_url}/markets/slug/{market_slug}"
        market_resp = requests.get(market_url)
        if market_resp.status_code != 200:
            print(f"  Failed to fetch market {market_slug}")
            continue

        market_data = market_resp.json()
        print("\nMarket:", market_slug)
        print("  Question:", market_data.get("question"))
        print("  Outcomes:", market_data.get("outcomes"))
        print("  Prices:", market_data.get("outcomePrices"))

if __name__ == "__main__":
    # Put the slug of the event from the frontend URL here:
    slug = "super-bowl-champion-2026-731"
    fetch_event_and_markets(slug)


Event data:
{'id': '23656', 'ticker': 'super-bowl-champion-2026-731', 'slug': 'super-bowl-champion-2026-731', 'title': 'Big Game Champion 2026', 'description': 'This is a market on predicting the winner of the NFL Super Bowl for the 2025/26 season. ', 'resolutionSource': '', 'startDate': '2025-05-01T20:30:33.749654Z', 'creationDate': '2025-05-01T20:30:33.749649Z', 'endDate': '2026-02-08T12:00:00Z', 'image': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/football-logo.png', 'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/football-logo.png', 'active': True, 'closed': True, 'archived': False, 'new': False, 'featured': False, 'restricted': True, 'volume': 704096822.722044, 'openInterest': 0, 'sortBy': 'price', 'createdAt': '2025-04-30T10:43:09.376079Z', 'updatedAt': '2026-02-10T08:25:39.858218Z', 'volume1wk': 45639071.63558505, 'volume1mo': 162843922.3865944, 'volume1yr': 704096822.7220505, 'enableOrderBook': True, 'negRisk': True, 'negRiskMarketID': '0x6a0d290c8ce1536

In [57]:
import requests
import pandas as pd
import json
import re

def get_event_markets(slug):
    base = "https://gamma-api.polymarket.com"
    resp = requests.get(f"{base}/events/slug/{slug}")
    resp.raise_for_status()
    return resp.json().get("markets", [])

def fetch_market(slug):
    base = "https://gamma-api.polymarket.com"
    resp = requests.get(f"{base}/markets/slug/{slug}")
    if resp.status_code != 200:
        return None
    return resp.json()

def parse_list(val):
    # Polymarket API sometimes returns stringified JSON arrays
    try:
        return json.loads(val) if isinstance(val, str) else val
    except json.JSONDecodeError:
        return val

def clean_team_name(question):
    """
    Remove boilerplate phrases so only the
    city/state + mascot remains (e.g., "Kansas City Chiefs").
    """
    if not isinstance(question, str):
        return question

    # Remove "Will the " at the beginning
    name = re.sub(r"^Will the\s+", "", question, flags=re.IGNORECASE)

    # Remove " win Super Bowl {year}?"
    name = re.sub(r"\s+win\s+Super Bowl\s+\d+\??$", "", name, flags=re.IGNORECASE)

    return name.strip()

def build_team_table(markets):
    rows = []

    for m in markets:
        slug = m.get("slug")
        detail = fetch_market(slug)
        if not detail:
            continue

        # Parse outcomes/prices arrays
        outcomes = parse_list(detail.get("outcomes", []))
        prices = parse_list(detail.get("outcomePrices", []))
        raw_vol = detail.get("volume", 0)

        # Convert volume
        try:
            volume = float(raw_vol)
        except ValueError:
            volume = 0.0

        # Find the YES outcome
        for outcome, price in zip(outcomes, prices):
            if outcome.lower() == "yes":
                try:
                    implied_pct = float(price) * 100
                except ValueError:
                    implied_pct = None

                # Clean the team name
                question_text = detail.get("question", slug)
                team_name = clean_team_name(question_text)

                rows.append({
                    "Team": team_name,
                    "ImpliedProbability(%)": implied_pct,
                    "VolumeUSD": round(volume, 2) 
                })
                break

    return pd.DataFrame(rows)

if __name__ == "__main__":
    event_slug = "super-bowl-champion-2026-731"
    markets = get_event_markets(event_slug)
    df = build_team_table(markets)
    print(df)


                          Team  ImpliedProbability(%)    VolumeUSD
0            Arizona Cardinals                    0.0  42048002.79
1              Atlanta Falcons                    0.0   9516223.35
2             Baltimore Ravens                    0.0   3538125.67
3                Buffalo Bills                    0.0   6163891.43
4            Carolina Panthers                    0.0  60236453.89
5                Chicago Bears                    0.0  10640593.55
6           Cincinnati Bengals                    0.0  39998043.16
7             Cleveland Browns                    0.0  51375598.94
8               Dallas Cowboys                    0.0  14853068.90
9               Denver Broncos                    0.0   9185427.81
10               Detroit Lions                    0.0   3076938.52
11           Green Bay Packers                    0.0   4023623.38
12              Houston Texans                    0.0   9002496.98
13          Indianapolis Colts                    0.0   725658

In [63]:
df.to_csv("/Users/Amelia/super_bowl_2026_polymarket.csv", index=False)